In [3]:
# Get network of a tweet
import requests
import json

url = "https://api-hoaxy.p.rapidapi.com/network"

headers = {
    'x-rapidapi-key': "6dda23c3b6msh40fb43640818777p173117jsn2e799b5b84a7",
    'x-rapidapi-host': "api-hoaxy.p.rapidapi.com"
    }

# ids must be an array, even if just 1 id
def get_network(ids, node_limit=1000, edge_limit=12500):
    querystring = {"ids":str(ids), "nodes_limit":str(node_limit), "edges_limit":str(edge_limit)}
    response = requests.request("GET", url, headers=headers, params=querystring)
    return json.loads(response.text)

print(get_network([64446]))

{'edges': [{'canonical_url': 'http://www.infowars.com/was-hillary-wearing-an-earpiece-during-last-nights-presidential-forum/', 'date_published': '2016-09-08T11:58:32.000Z', 'domain': 'infowars.com', 'from_user_id': '15444711', 'from_user_screen_name': 'FloridaGOP', 'id': 64446, 'is_mention': False, 'site_type': 'claim', 'title': 'Was Hillary Wearing an Earpiece During Last Night’s Presidential Forum?', 'to_user_id': '330487457', 'to_user_screen_name': 'pattiewhi1', 'tweet_created_at': '2016-09-08T18:28:57.000Z', 'tweet_id': '773951272294555648', 'tweet_type': 'retweet', 'url_id': 1272606}, {'canonical_url': 'http://www.infowars.com/was-hillary-wearing-an-earpiece-during-last-nights-presidential-forum/', 'date_published': '2016-09-08T11:58:32.000Z', 'domain': 'infowars.com', 'from_user_id': '15444711', 'from_user_screen_name': 'FloridaGOP', 'id': 64446, 'is_mention': False, 'site_type': 'claim', 'title': 'Was Hillary Wearing an Earpiece During Last Night’s Presidential Forum?', 'to_user